In [1]:
%%time
import sys
if not r'G:\PythonProjects\WineRecognition2' in sys.path:
    sys.path.insert(0, r'G:\PythonProjects\WineRecognition2')

import os
import json
import torch
import numpy as np
import pandas as pd
import sklearn_crfsuite
import eli5
from sklearn.model_selection import train_test_split

from data_master import DataGenerator, DataLoader, count_unk_foreach_tag, compute_model_confidence
from mlflow_utils import log_mlflow_on_train
from features import features

Wall time: 1.95 s


In [2]:
%%time
DATASET_PATH = r"G:\PythonProjects\WineRecognition2\data\text\Halliday_Wine_AU-only_completed_rows-complex.txt"
LSTM_MODEL_PATH = r"G:\PythonProjects\WineRecognition2\artifacts\train\BiLSTM_CRF_10112021_030733\model\data\model.pth"
VOCAB_PATH = r"G:\PythonProjects\WineRecognition2\data\vocabs\Words_Halliday_Wine_AU.json"
CASE_SENSITIVE_VOCAB = False
DICTIONARY_PATH = r"G:\PythonProjects\WineRecognition2\data\dictionaries\Dict-byword_Halliday_Winesearcher_Wine_AU-only_completed_rows"
DATAINFO_PATH = 'G:/PythonProjects/WineRecognition2/data_info.json'
DEVICE = 'cuda'
MODEL_NAME = "CRF_with_LSTM_features"
ALGORITHM = 'lbfgs'
C1 = 0.1
C2 = 0.1
MAX_ITERATIONS = 5
ALL_POSSIBLE_TRANSITIONS = True
TEST_SIZE = 0.2
RUN_NAME = 'Train-LSTMfeatures'
OUTPUT_DIR = r"G:\PythonProjects\WineRecognition2\artifacts\train\test_exp"
START_TIME = ''

Wall time: 0 ns


In [3]:
%%time
model = torch.load(LSTM_MODEL_PATH).to(DEVICE).eval()
freq_dict = DataLoader.load_frequency_dictionary(DICTIONARY_PATH)
with open(VOCAB_PATH, encoding='utf-8') as file:
    word_to_ix = json.load(file)
if not CASE_SENSITIVE_VOCAB:
    word_to_ix = {word.lower(): index for word, index in word_to_ix.items()}

Wall time: 1.65 s


In [4]:
%%time
# getting features from lstm

def get_lstm_features(model, x):
    x = model.embedding(x)
    x, _ = model.lstm(x)
    return x

def features_with_keys(sentence):
    return [{f'A{i}': feature for i, feature in enumerate(features)} for features in sentence]

unk = 'UNK' if CASE_SENSITIVE_VOCAB else 'unk'
X_tensors = []

with torch.no_grad():
    sents = DataGenerator.generate_sents2(open(DATASET_PATH, encoding='utf-8').read().split('\n'))
    filename = os.path.splitext(os.path.split(DATASET_PATH)[-1])[0]
    metadata = {'features': [], 'labels': []}

    for x_sent, y_sent in sents:
        our_features = features.sent2features(list(zip(x_sent, y_sent)), freq_dict)
        
        if not CASE_SENSITIVE_VOCAB:
            x_sent = [word.lower() for word in x_sent]
            
        x_tensor = torch.tensor(
            [word_to_ix[word] if word in word_to_ix else word_to_ix[unk] for word in x_sent],
            dtype=torch.int64
        )
        X_tensors.append(x_tensor)
        
        final_features = get_lstm_features(model, x_tensor.to(DEVICE).unsqueeze(0))
        final_features = features_with_keys(final_features.squeeze(0).detach().cpu().numpy())
        for i in range(len(x_sent)):
            final_features[i].update(our_features[i])
        metadata['features'].append(final_features)
        metadata['labels'].append(y_sent)
        
labels = metadata['labels']
x = metadata['features']
len(x), len(labels)

Wall time: 26.4 s


(100, 100)

In [5]:
%%time
X_train, X_val, y_train, y_val, train_sents, val_sents = train_test_split(x, labels, sents, test_size=TEST_SIZE)
len(X_train), len(X_val), len(y_train), len(y_val), len(train_sents), len(val_sents)

Wall time: 0 ns


(80, 20, 80, 20, 80, 20)

In [6]:
%%time
model = sklearn_crfsuite.CRF(
    algorithm=ALGORITHM,
    c1=C1,
    c2=C2,
    max_iterations=MAX_ITERATIONS,
    all_possible_transitions=ALL_POSSIBLE_TRANSITIONS
)
model.fit(X_train, y_train)
eli5.show_weights(model, top=(30, 30))

Wall time: 263 ms


D:\ProgramFiles\anaconda3\envs\wine-recognition\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


From \ To,Add_BottleSize,Add_Brand,Add_ClosureType,Add_GeoIndication,Add_GrapeVarieties,Add_KeyWordTrue,Add_Price,Add_Sweetness,Add_TradeName,Add_Vintage,Add_WineColor,Add_WineType,Punctuation
Add_BottleSize,-0.048,-0.029,0.025,-0.027,-0.038,-0.025,0.066,0.024,-0.043,0.001,0.047,0.008,-0.022
Add_Brand,-0.033,0.144,-0.046,0.01,0.055,0.122,-0.036,-0.035,-0.194,-0.024,-0.018,-0.04,0.0
Add_ClosureType,-0.003,-0.033,0.196,-0.032,-0.054,-0.026,0.007,-0.025,-0.046,0.016,0.017,0.029,-0.024
Add_GeoIndication,-0.019,-0.062,-0.032,0.185,0.083,-0.033,-0.037,0.021,-0.072,-0.027,0.016,-0.017,-0.028
Add_GrapeVarieties,0.027,-0.09,-0.003,0.017,0.056,-0.002,-0.042,0.088,-0.078,-0.026,0.012,0.022,0.067
Add_KeyWordTrue,-0.0,-0.026,-0.004,0.035,0.034,0.158,-0.022,-0.02,-0.039,-0.016,-0.021,-0.026,-0.014
Add_Price,-0.026,-0.025,-0.044,-0.021,-0.035,-0.015,-0.031,-0.022,-0.039,-0.021,-0.025,-0.029,-0.015
Add_Sweetness,0.026,-0.027,0.003,-0.029,-0.041,-0.024,0.069,-0.034,-0.052,0.019,0.088,0.056,-0.021
Add_TradeName,-0.035,0.114,-0.039,-0.079,-0.107,-0.033,-0.037,-0.033,0.375,-0.026,-0.036,-0.037,0.13
Add_Vintage,-0.032,-0.032,-0.032,-0.022,-0.027,-0.021,0.266,-0.023,0.231,-0.026,-0.029,-0.033,-0.021


In [7]:
%%time
y_pred = model.predict(X_val)
marginals = model.predict_marginals(X_val)

Wall time: 10 ms


In [8]:
with open(DATAINFO_PATH) as file:
    unk_foreach_tag = count_unk_foreach_tag(X_tensors, labels, json.load(file)['keys']['all'], word_to_ix[unk])

In [11]:
confs = compute_model_confidence(marginals)
prob_table = DataGenerator.generate_probability_table(marginals, val_sents)

In [14]:
%%time
test_eval = [list(zip(sentence, tags, y_pred[index])) for index, (sentence, tags) in enumerate(val_sents)]

Wall time: 999 µs


In [15]:
%%time
run_params = {
    'dataset_path': DATASET_PATH,
    'lstm_model_path': LSTM_MODEL_PATH,
    'vocab_path': VOCAB_PATH,
    'case_sensitive_vocab': CASE_SENSITIVE_VOCAB,
    'dictionary_path': DICTIONARY_PATH,
    'datainfo_path': DATAINFO_PATH,
    'device': DEVICE,
    'model_name': MODEL_NAME,
    'algorithm': ALGORITHM,
    'c1': C1,
    'c2': C2 ,
    'max_iterations': MAX_ITERATIONS,
    'all_possible_transitions': ALL_POSSIBLE_TRANSITIONS,
    'test_size': TEST_SIZE,
    'runname': RUN_NAME,
    'start_time': START_TIME,
    'output_dir': OUTPUT_DIR,
    'models_confidence': np.mean(confs),
    'unk_foreach_tag': json.dumps(unk_foreach_tag),
    'prob_table': prob_table
}

Wall time: 0 ns


In [ ]:
%%time
log_mlflow_on_train(
    run_params=run_params,
    model=model,
    y_true=y_val,
    y_pred=y_pred,
    test_eval=test_eval
)